# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [2]:
# Set the file path to import the WeatherPy_database.csv file.
file_path = "../Weather_Database/WeatherPy_database.csv"

# Load the CSV file into a Pandas DataFrame.
city_data_df = pd.read_csv(file_path)

# Display sample data.
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Halifax,CA,44.6453,-63.5724,67.89,84,40,10.36,fog
1,1,Arraial Do Cabo,BR,-22.9661,-42.0278,73.15,86,20,16.78,few clouds
2,2,Saint Anthony,US,45.0205,-93.2180,39.24,45,40,17.27,scattered clouds
3,3,Tutoia,BR,-2.7619,-42.2744,83.12,72,10,15.32,clear sky
4,4,Carnarvon,AU,-24.8667,113.6333,63.97,62,0,20.24,clear sky
5,5,Morondava,MG,-20.2833,44.2833,77.43,78,5,14.72,clear sky
6,6,Iqaluit,CA,63.7506,-68.5145,32.70,69,100,15.99,overcast clouds
7,7,Hermanus,ZA,-34.4187,19.2345,63.81,95,62,3.89,broken clouds
8,8,Requena,ES,39.4883,-1.1004,63.79,85,97,4.25,overcast clouds
9,9,Gamboula,CF,4.1178,15.1393,69.85,94,88,0.87,overcast clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [3]:
# Prompt the user to enter the minimum temperature criteria.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))

# Prompt the user to enter the maximum temperature criteria.
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [4]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) &
                                       (city_data_df["Max Temp"] <= max_temp)]
                                    
# Display sample data.
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Arraial Do Cabo,BR,-22.9661,-42.0278,73.15,86,20,16.78,few clouds
3,3,Tutoia,BR,-2.7619,-42.2744,83.12,72,10,15.32,clear sky
5,5,Morondava,MG,-20.2833,44.2833,77.43,78,5,14.72,clear sky
11,11,Umarkot,PK,25.3616,69.7362,82.44,24,0,9.91,clear sky
13,13,Souillac,MU,-20.5167,57.5167,70.63,72,53,10.40,broken clouds
14,14,Mamallapuram,IN,12.6264,80.1722,84.25,77,100,3.02,overcast clouds
19,19,Saint-Philippe,RE,-21.3585,55.7679,71.19,84,43,9.46,scattered clouds
20,20,Cururupu,BR,-1.8283,-44.8683,86.63,57,26,7.14,scattered clouds
21,21,Thiruvananthapuram,IN,8.4833,76.9167,78.78,89,75,0.00,light rain
24,24,Ilhabela,BR,-23.7781,-45.3581,73.31,80,100,6.08,overcast clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [5]:
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
clean_travel_cities = preferred_cities_df.dropna()

# Display sample data.
clean_travel_cities.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Arraial Do Cabo,BR,-22.9661,-42.0278,73.15,86,20,16.78,few clouds
3,3,Tutoia,BR,-2.7619,-42.2744,83.12,72,10,15.32,clear sky
5,5,Morondava,MG,-20.2833,44.2833,77.43,78,5,14.72,clear sky
11,11,Umarkot,PK,25.3616,69.7362,82.44,24,0,9.91,clear sky
13,13,Souillac,MU,-20.5167,57.5167,70.63,72,53,10.40,broken clouds
14,14,Mamallapuram,IN,12.6264,80.1722,84.25,77,100,3.02,overcast clouds
19,19,Saint-Philippe,RE,-21.3585,55.7679,71.19,84,43,9.46,scattered clouds
20,20,Cururupu,BR,-1.8283,-44.8683,86.63,57,26,7.14,scattered clouds
21,21,Thiruvananthapuram,IN,8.4833,76.9167,78.78,89,75,0.00,light rain
24,24,Ilhabela,BR,-23.7781,-45.3581,73.31,80,100,6.08,overcast clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [6]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# Display sample data.
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng
1,Arraial Do Cabo,BR,73.15,few clouds,-22.9661,-42.0278
3,Tutoia,BR,83.12,clear sky,-2.7619,-42.2744
5,Morondava,MG,77.43,clear sky,-20.2833,44.2833
11,Umarkot,PK,82.44,clear sky,25.3616,69.7362
13,Souillac,MU,70.63,broken clouds,-20.5167,57.5167
14,Mamallapuram,IN,84.25,overcast clouds,12.6264,80.1722
19,Saint-Philippe,RE,71.19,scattered clouds,-21.3585,55.7679
20,Cururupu,BR,86.63,scattered clouds,-1.8283,-44.8683
21,Thiruvananthapuram,IN,78.78,light rain,8.4833,76.9167
24,Ilhabela,BR,73.31,overcast clouds,-23.7781,-45.3581


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [7]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame.
hotel_df["Hotel Name"] = ""

# Display sample data.
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Arraial Do Cabo,BR,73.15,few clouds,-22.9661,-42.0278,
3,Tutoia,BR,83.12,clear sky,-2.7619,-42.2744,
5,Morondava,MG,77.43,clear sky,-20.2833,44.2833,
11,Umarkot,PK,82.44,clear sky,25.3616,69.7362,
13,Souillac,MU,70.63,broken clouds,-20.5167,57.5167,
14,Mamallapuram,IN,84.25,overcast clouds,12.6264,80.1722,
19,Saint-Philippe,RE,71.19,scattered clouds,-21.3585,55.7679,
20,Cururupu,BR,86.63,scattered clouds,-1.8283,-44.8683,
21,Thiruvananthapuram,IN,78.78,light rain,8.4833,76.9167,
24,Ilhabela,BR,73.31,overcast clouds,-23.7781,-45.3581,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [8]:
# Review the parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [9]:
# Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude as parameters to the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search.
    hotels = requests.get(base_url, params = params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# Display sample data.
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Arraial Do Cabo,BR,73.15,few clouds,-22.9661,-42.0278,Pousada Porto Praia
3,Tutoia,BR,83.12,clear sky,-2.7619,-42.2744,Pousada Guará's
5,Morondava,MG,77.43,clear sky,-20.2833,44.2833,La Case Bambou
11,Umarkot,PK,82.44,clear sky,25.3616,69.7362,Al-Qamar Boys Hostel
13,Souillac,MU,70.63,broken clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
14,Mamallapuram,IN,84.25,overcast clouds,12.6264,80.1722,Radisson Blu Resort Temple Bay Mamallapuram
19,Saint-Philippe,RE,71.19,scattered clouds,-21.3585,55.7679,Le four à pain
20,Cururupu,BR,86.63,scattered clouds,-1.8283,-44.8683,POUSADA TRAVÉS'CIA
21,Thiruvananthapuram,IN,78.78,light rain,8.4833,76.9167,Hotel Residency Tower
24,Ilhabela,BR,73.31,overcast clouds,-23.7781,-45.3581,Ilha Flat Hotel


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [11]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.drop(hotel_df[hotel_df["Hotel Name"] == ""].index)

# Display sample data.
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Arraial Do Cabo,BR,73.15,few clouds,-22.9661,-42.0278,Pousada Porto Praia
3,Tutoia,BR,83.12,clear sky,-2.7619,-42.2744,Pousada Guará's
5,Morondava,MG,77.43,clear sky,-20.2833,44.2833,La Case Bambou
11,Umarkot,PK,82.44,clear sky,25.3616,69.7362,Al-Qamar Boys Hostel
13,Souillac,MU,70.63,broken clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
14,Mamallapuram,IN,84.25,overcast clouds,12.6264,80.1722,Radisson Blu Resort Temple Bay Mamallapuram
19,Saint-Philippe,RE,71.19,scattered clouds,-21.3585,55.7679,Le four à pain
20,Cururupu,BR,86.63,scattered clouds,-1.8283,-44.8683,POUSADA TRAVÉS'CIA
21,Thiruvananthapuram,IN,78.78,light rain,8.4833,76.9167,Hotel Residency Tower
24,Ilhabela,BR,73.31,overcast clouds,-23.7781,-45.3581,Ilha Flat Hotel


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [ ]:
# In case of an empty DataFrame, load the sample data provided.
# clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
# clean_hotel_df.head(10)

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [12]:
# Set the file name.
output_data_file = "WeatherPy_vacation.csv"

# Create a CSV file by using the clean_hotel_df DataFrame.
clean_hotel_df.to_csv(output_data_file)

### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [13]:
# Review the formatting template provided.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [14]:
# Get the data from each row in the clean_hotel_df DataFrame,
# add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data.
hotel_info[:10]

['\n<dl>\n<dt>Hotel Name</dt><dd>Pousada Porto Praia</dd>\n<dt>City</dt><dd>Arraial Do Cabo</dd>\n<dt>Country</dt><dd>BR</dd>\n<dt>Current Weather</dt><dd>few clouds and 73.15 °F</dd>\n</dl>\n',
 "\n<dl>\n<dt>Hotel Name</dt><dd>Pousada Guará's</dd>\n<dt>City</dt><dd>Tutoia</dd>\n<dt>Country</dt><dd>BR</dd>\n<dt>Current Weather</dt><dd>clear sky and 83.12 °F</dd>\n</dl>\n",
 '\n<dl>\n<dt>Hotel Name</dt><dd>La Case Bambou</dd>\n<dt>City</dt><dd>Morondava</dd>\n<dt>Country</dt><dd>MG</dd>\n<dt>Current Weather</dt><dd>clear sky and 77.43 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Al-Qamar Boys Hostel</dd>\n<dt>City</dt><dd>Umarkot</dd>\n<dt>Country</dt><dd>PK</dd>\n<dt>Current Weather</dt><dd>clear sky and 82.44 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Shanti Maurice Resort & Spa</dd>\n<dt>City</dt><dd>Souillac</dd>\n<dt>Country</dt><dd>MU</dd>\n<dt>Current Weather</dt><dd>broken clouds and 70.63 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Radisson Blu Resort Te

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [15]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data.
locations.head(10)

,Lat,Lng
1,-22.9661,-42.0278
3,-2.7619,-42.2744
5,-20.2833,44.2833
11,25.3616,69.7362
13,-20.5167,57.5167
14,12.6264,80.1722
19,-21.3585,55.7679
20,-1.8283,-44.8683
21,8.4833,76.9167
24,-23.7781,-45.3581


## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [16]:
# Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Create a figure to add the Google map as a layer.
fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 1.5)
fig.add_layer(marker_layer)

# Display the figure containing the map.
fig

Figure(layout=FigureLayout(height='420px'))